In [38]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster
import pandas as pd
import hvplot.dask
import re


#code found on stack exchange
#makes a new client if one is not running, runs 4 workers
try:
    client = Client('tcp://localhost:8787', timeout='2s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)
#

client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [39]:
keyWords = [
    'mental health', 
    'depression',
    'depressed', 
    'anxiety',
    'anxious', 
    'ADHD', 
    'attention deficit',
    'OCD',
    'obsessive-compulsive disorder',
    'PTSD',
    'posttraumatic stress disorder',
    'trauma',
    'bipolar',
    'autism',
    'schizophrenia',
    'eating disorder',
    'anorexia',
    'bulimia',
    'psychosis',
    'psychologist',
    'psychotherapist',
    'psychiatrist',
    'psychotherapy',
    'depressant'
]

In [40]:
#read and print the dataframes

#contains tweets (and time of the tweet) that contain mental health key word
#polled ever 1 minute
tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t' )

#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='mental_health_tweet_count.csv', sep='\t')

print(tweetdf)

print(countdf)

Dask DataFrame Structure:
                tweet    time
npartitions=1                
               object  object
                  ...     ...
Dask Name: read-csv, 1 tasks
Dask DataFrame Structure:
              timestamp  count
npartitions=1                 
                 object  int64
                    ...    ...
Dask Name: read-csv, 1 tasks


In [41]:

#specify column types
tweetdf['time'] = dd.to_datetime(tweetdf['time'])
countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

In [42]:
#sort the dataframes based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])
tweetdf = tweetdf.sort_values(ascending=True, by=['month', 'day', 'hour', 'minute'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

# countdf = client.persist(countdf)

In [43]:
# countdf.hvplot(x='ts_display', y='count').opts(xrotation=90, fontscale=0.5)

In [44]:
#pre process

def preprocess(text):
    text = text[1]
    text = text.lower()
    return re.sub(string=text, pattern='[^a-z ]', repl='')

tweetdf['text'] = tweetdf['tweet'].map_partitions(preprocess)

In [45]:
#determine key words in tweet
def containsKey(text, key):
    return text.str.contains(key)

# create a new column for each key, with values of whether or not
# the tweet in that row contains that key
for i in keyWords:
    tweetdf[i] = tweetdf['tweet'].map_partitions(containsKey, i)


In [46]:
#count tweets each keyword appears in
#if a tweet contains multiple keywords, then they are added 
#to the result df as |word1|word2| etc.

def keysInTweet(df):
    keys = ''
    for k in keyWords:
        if df[k]:
            keys = keys + '|' + k
    keys = keys + '|'
    return keys

tweetdf['keys'] = tweetdf.apply(keysInTweet, axis=1, meta=('keys', 'object'))

# convert the value counts to its own df
multiKeyCount = tweetdf['keys'].value_counts().to_frame()
# REMOVE? drop rows that dont have a key in the tweet
multiKeyCount = multiKeyCount.loc[multiKeyCount['keys'] != '']
# copy the key column (which actually holds counts), rename it and then drop the original
multiKeyCount['count'] = multiKeyCount['keys']
multiKeyCount = multiKeyCount.drop(labels='keys', axis=1)

# multiKeyCount = client.persist(multiKeyCount)

In [47]:
# multiKeyCount.hvplot.bar(y='count', use_index=True)

In [48]:
# create an empty dask dataframe, with 1 parition (since max rows is 24)
keyCount = pd.DataFrame({'key' : keyWords})
keyCount = dd.from_pandas(keyCount, npartitions=1)

# given key k, find the number of times k appears in a tweet
def countKeys(k):
    return tweetdf[k].sum().compute()

# create a new row of the key count
keyCount['count'] = keyCount['key'].map(countKeys, meta=('count', 'int64'))

# sort rows by descending count
keyCount = keyCount.sort_values(by='count', ascending=True)

keyCount = client.persist(keyCount)

In [49]:
keyCountPlot = keyCount.hvplot.bar(y='count', x='key').opts(fontscale=0.75, invert_axes=True)
hvplot.save(keyCountPlot, 'keyCountPlot.png')